# Demo of Translation Use Case with Lumigator SDK

### Pre-requisites
1. To use this notebook, you'll need to have Lumigator running. In a terminal, run:
```
git clone git@github.com:mozilla-ai/lumigator.git
cd lumigator
make start-lumigator-build
```

2. Follow the [README.md](https://github.com/mozilla-ai/lumigator/tree/main/notebooks#readme) to setup your Python environment. Then attach this notebook to the corresponding kernel.

### Lumigator Client
Check if client is up and running

In [3]:
from lumigator_sdk.lumigator import LumigatorClient

In [4]:
LUMI_HOST = "localhost:8000"
client = LumigatorClient(api_host=LUMI_HOST)
print(f"Connection is: {client.health.healthcheck().status}")

Connection is: OK


### Dataset
Using `google/wmt24pp` datasets from Huggingface Datasets, released as a part of [WMT24++: Expanding the Language Coverage of WMT24 to 55 Languages & Dialects](https://arxiv.org/abs/2502.12404v1)

In [5]:
import pandas as pd
from datasets import load_dataset

pd.set_option("display.max_colwidth", None)

In [6]:
source_language = "English"
target_language = "German"
language_pair = "en-de_DE"
dataset = load_dataset("google/wmt24pp", language_pair, split="train")

In [7]:
df_translation = pd.DataFrame(dataset[1:])[["source", "target"]]
print(f"Loaded {len(df_translation)} sentence pairs")
df_translation.head(3)

Loaded 997 sentence pairs


,source,target
0,"Siso's depictions of land, water center new gallery exhibition",Sisos Darstellungen von Land und Wasser in neuer Ausstellung
1,"""People Swimming in the Swimming Pool"" from 2022 is one Vicente Siso artwork that will display at Tierra del Sol Gallery beginning Jan. 13. (photo courtesy of Vicente Siso)","„People Swimming in the Swimming Pool“ aus dem Jahr 2022 ist ein Kunstwerk von Vicente Siso, das ab dem 13. Januar in der Galerie „Tierra del Sol“ ausgestellt wird (Foto bereitgestellt von Vicente Siso)"
2,"Tierra del Sol is pleased to present ""Vicente Siso: Memories of the Land and Water"" at the new gallery location in West Hollywood. Siso has been an artist in the Studio Arts Program since 2012, this marks his debut solo exhibition. Siso was born 1962 in Madrid and raised between Venezuela, Trinidad and Miami; he moved with his family to Southern California in his early 20s.","„Vicente Siso: Memories of the Land and Water“ – das ist der Titel der Ausstellung im neuen Standort der Galerie „Tierra del Sol“ in West Hollywood. Siso ist seit 2012 als Künstler im Studio Arts Progam dabei und zeigt jetzt seine erste Solo-Ausstellung. Siso wurde 1962 in Madrid geboren und ist in Venezuela, Trinidad und Miami aufgewachsen. Anfang zwanzig zog er mit seiner Familie nach Südkalifornien."


In [8]:
# Rename the columns to examples and ground_truth
df_translation = df_translation.rename(columns={"source": "examples", "target": "ground_truth"})
filename = "translation_eval_dataset.csv"
df_translation = df_translation.head(10)
df_translation.to_csv(filename, index=False)  # Save the file locally with 10 samples

### Upload Dataset

In [9]:
from pathlib import Path

from lumigator_schemas.datasets import DatasetFormat

# Upload that file that we created earlier
with Path.open(Path(filename), "r") as file:
    data = file.read()
dataset_response = client.datasets.create_dataset(dataset=data, format=DatasetFormat.JOB)
dataset_id = dataset_response.id
print(f"Dataset uploaded and has ID: {dataset_id}")

Dataset uploaded and has ID: 4a5d9fa0-5ce7-440d-8b9d-0faf65c18616


### Create Experiment

In [10]:
from lumigator_schemas.experiments import ExperimentCreate

# Task as translation with necessary params
task_definition = {
    "task": "translation",
    "source_language": source_language,
    "target_language": target_language,
}
max_samples = 2

request = ExperimentCreate(
    name=f"GenAI in Localization Event Demo {source_language} to {target_language}",
    description="Experiment for demo",
    dataset=dataset_id,
    task_definition=task_definition,
    max_samples=max_samples,
)

experiment_response = client.experiments.create_experiment(request)
experiment_id = experiment_response.id
print(f"Experiment created and has ID: {experiment_id}")

Experiment created and has ID: 8


### Create and Run Workflows
- One with API-based Open AI GPT-4o-mini
- One with locally hosted Ollama model (Pre-requisite: Execute `ollama run qwen2.5` in a terminal window)
- One with a bilingual Opus-MT model on HuggingFace

G-Eval: NLG Evaluation using GPT-4 with Better Human Alignment
https://arxiv.org/abs/2303.16634

In [11]:
from lumigator_schemas.workflows import WorkflowCreateRequest

custom_system_prompt = f"""
You are an expert in {source_language} and {target_language}. 
Please provide a high-quality translation of the following text from {source_language} to {target_language}.
Only generate the translated text. No additional text or explanation needed.
"""

batch_size = 2

metrics = [
    "bleu",
    "meteor",
    "comet",
    # LLM-as-a-judge metrics:
    # Configure "llm_as_judge" argument if a judge is a different model
    # See: https://github.com/mozilla-ai/lumigator/pull/1321
    "g_eval_translation",
    "g_eval_translation_noref",
]

configurations = [
    # OpenAI GPt-4o-mini no explicit system prompt - uses default prompt set by Lumigator under the hood
    {
        "name": "Translation with gpt-4o-mini",
        "model": "gpt-4o-mini",
        "provider": "openai",
        "secret_key_name": "openai_api_key",
    },
    # Locally hosted Ollama model with custom system prompt
    {
        "name": "Translation with ollama qwen2.5",
        "model": "qwen2.5",
        "provider": "openai",
        "base_url": "http://localhost:11434/v1",
        "system_prompt": custom_system_prompt,
        "secret_key_name": "openai_api_key",
    },
    # HuggingFace Opus-MT bi-lingual model
    {
        "name": "Translation with Helsinki-NLP/opus-mt",
        "model": "Helsinki-NLP/opus-mt-en-de",
        "provider": "hf",
    },
]

workflow_ids = []
for config in configurations:
    params = {
        "name": config["name"],
        "model": config["model"],
        "experiment_id": experiment_id,
        "batch_size": batch_size,
        "metrics": metrics,
    }
    params.update(config)
    request = WorkflowCreateRequest(**params)
    created_workflow = client.workflows.create_workflow(request)
    print(f"Created workflow {created_workflow.name} with ID {created_workflow.id} for model {created_workflow.model}")
    workflow_ids.append(created_workflow.id)

Created workflow Translation with gpt-4o-mini with ID 43bf2310542842baa33cde0e6b7c948c for model gpt-4o-mini
Created workflow Translation with ollama qwen2.5 with ID d1c170dfc33d415c8e77c0bd7a6dcd5d for model qwen2.5
Created workflow Translation with Helsinki-NLP/opus-mt with ID ffd9de4984ea4968aa55fb46f8d9f4b6 for model Helsinki-NLP/opus-mt-en-de


### View Results
Check http://localhost/experiments for status. 

__Once the experiment status is `SUCCEEDED`, proceed further.__

In [12]:
from time import sleep

from lumigator_schemas.workflows import WorkflowStatus

still_running = True
while still_running:
    experiment_details = client.experiments.get_experiment(experiment_id)
    still_running_workflows = [
        workflow.name
        for workflow in experiment_details.workflows
        if workflow.status not in [WorkflowStatus.SUCCEEDED, WorkflowStatus.FAILED]
    ]
    still_running = bool(still_running_workflows)
    if still_running:
        print("Workflows still running ...")
        sleep(60)

workflow_results = [client.workflows.get_workflow(workflow_id) for workflow_id in workflow_ids]
df_metrics = pd.concat([pd.DataFrame.from_dict({wr.model: wr.metrics}).T for wr in workflow_results])
df_metrics[
    [
        "meteor_meteor_mean",
        "bleu_bleu_mean",
        "comet_mean_score",
        "g_eval_translation_consistency_mean",
        "g_eval_translation_fluency_mean",
        "g_eval_translation_noref_consistency_mean",
        "g_eval_translation_noref_fluency_mean",
    ]
]

Workflows still running ...
Workflows still running ...
Workflows still running ...


,meteor_meteor_mean,bleu_bleu_mean,comet_mean_score,g_eval_translation_consistency_mean,g_eval_translation_fluency_mean,g_eval_translation_noref_consistency_mean,g_eval_translation_noref_fluency_mean
gpt-4o-mini,0.672,0.448,0.599,0.783,0.832,0.998,0.944
qwen2.5,0.598,0.362,0.466,0.774,0.659,0.983,0.872
Helsinki-NLP/opus-mt-en-de,0.625,0.146,0.516,0.652,0.558,0.818,0.680


In [14]:
output_texts_per_model = {}
for wr in workflow_results:
    # Input and predictions
    base_dict = pd.read_json(wr.artifacts_download_url)["artifacts"][
        ["examples", "ground_truth", "predictions"]
    ].to_dict()

    # LLM as a Judge Metrics
    base_dict["g_eval_consistency"] = pd.read_json(wr.artifacts_download_url)["metrics"][
        ["g_eval_translation"]
    ].to_dict()["g_eval_translation"]["consistency"]
    base_dict["g_eval_fluency"] = pd.read_json(wr.artifacts_download_url)["metrics"][["g_eval_translation"]].to_dict()[
        "g_eval_translation"
    ]["fluency"]
    output_texts_per_model[wr.model] = base_dict

for model_name, df_texts in output_texts_per_model.items():
    print(f"Model: {model_name}")
    display(pd.DataFrame(df_texts).head(2))

Model: gpt-4o-mini


,examples,ground_truth,predictions,g_eval_consistency,g_eval_fluency
0,"Siso's depictions of land, water center new gallery exhibition",Sisos Darstellungen von Land und Wasser in neuer Ausstellung,Sisos Darstellungen von Land und Wasser stehen im Mittelpunkt der neuen Galerieausstellung.,"{'score': 0.783501935092053, 'reason': 'The Actual Output closely matches the Expected Output but includes 'stehen im Mittelpunkt', adding emphasis on the centrality of the depictions in the exhibition not mentioned in the Expected Output.'}","{'score': 0.8507981545435841, 'reason': 'The translation is mostly accurate and follows the structure closely, with no grammatical or spelling errors. However, the phrase 'stehen im Mittelpunkt der neuen Galerieausstellung' adds 'stehen im Mittelpunkt der' which is not present in the expected output, slightly affecting fluency.'}"
1,"""People Swimming in the Swimming Pool"" from 2022 is one Vicente Siso artwork that will display at Tierra del Sol Gallery beginning Jan. 13. (photo courtesy of Vicente Siso)","„People Swimming in the Swimming Pool“ aus dem Jahr 2022 ist ein Kunstwerk von Vicente Siso, das ab dem 13. Januar in der Galerie „Tierra del Sol“ ausgestellt wird (Foto bereitgestellt von Vicente Siso)","""Menschen, die im Schwimmbad schwimmen"" aus dem Jahr 2022 ist ein Kunstwerk von Vicente Siso, das ab dem 13. Januar in der Tierra del Sol Gallery ausgestellt wird. (Foto mit freundlicher Genehmigung von Vicente Siso)","{'score': 0.809247416736514, 'reason': 'The translation is mostly accurate but deviates by translating the title into German, which doesn't remain consistent with the Expected Output. The gallery name is translated incorrectly and the photo credit phrasing is also slightly off.'}","{'score': 0.7965320893502961, 'reason': 'The translation is fluent with minor deviations in word choice; 'Menschen, die im Schwimmbad schwimmen' deviates from 'People Swimming in the Swimming Pool' by translating rather than keeping the title as is. Additionally, 'Tierra del Sol Gallery' was not translated to 'Galerie', and 'Foto mit freundlicher Genehmigung' is slightly different from 'Foto bereitgestellt von'. Overall, the translation is mostly clear and easy to understand.'}"


Model: qwen2.5


,examples,ground_truth,predictions,g_eval_consistency,g_eval_fluency
0,"Siso's depictions of land, water center new gallery exhibition",Sisos Darstellungen von Land und Wasser in neuer Ausstellung,Sisos Darstellungen von Land und Wasser bilden den Mittelpunkt der neuen Galerieausstellung,"{'score': 0.7584223499454361, 'reason': 'The actual output correctly identifies 'Sisos Darstellungen von Land und Wasser' as the main subject, similar to the expected output. However, it adds 'bilden den Mittelpunkt der neuen Galerieausstellung', which is not explicitly stated in the expected output, thus introducing additional information.'}","{'score': 0.77087828225066, 'reason': 'The actual output uses 'bilden den Mittelpunkt der neuen Galerieausstellung' instead of 'in neuer Ausstellung,' which adds more detail than needed but remains grammatically correct and clear. The translation is natural and easy to understand, with minor differences that do not significantly affect the fluency.'}"
1,"""People Swimming in the Swimming Pool"" from 2022 is one Vicente Siso artwork that will display at Tierra del Sol Gallery beginning Jan. 13. (photo courtesy of Vicente Siso)","„People Swimming in the Swimming Pool“ aus dem Jahr 2022 ist ein Kunstwerk von Vicente Siso, das ab dem 13. Januar in der Galerie „Tierra del Sol“ ausgestellt wird (Foto bereitgestellt von Vicente Siso)","""Leute, die im Schwimmbad schwimmen,"" von 2022 ist ein Werk des Künstlers Vicente Siso, das ab dem 13. Januar im Tierra del Sol Gallery ausgestellt sein wird. (Foto courtoisie von Vicente Siso)","{'score': 0.7911382512918791, 'reason': 'The translation closely follows the expected output with minor deviations: 1) The phrase 'People Swimming in the Swimming Pool' was correctly retained in English, 2) 'photo courtoisie von Vicente Siso' introduces a slight inconsistency with 'Foto bereitgestellt von Vicente Siso,' and 3) 'Tierra del Sol Gallery' instead of 'Galerie 'Tierra del Sol'' shows a slight variation in noun phrase structure. Otherwise, factual details are accurately translated.'}","{'score': 0.5433600169040791, 'reason': 'The translation is generally understandable but has a few issues: 'Leute, die im Schwimmbad schwimmen' does not maintain the original English title's non-translated style; 'im Tierra del Sol Gallery' should align with 'in der Galerie „Tierra del Sol“'; and 'courtoisie' is a non-standard term in this context compared to 'bereitgestellt'.'}"


Model: Helsinki-NLP/opus-mt-en-de


,examples,ground_truth,predictions,g_eval_consistency,g_eval_fluency
0,"Siso's depictions of land, water center new gallery exhibition",Sisos Darstellungen von Land und Wasser in neuer Ausstellung,"Siso's Darstellungen von Land, Wasserzentrum neue Galerie Ausstellung","{'score': 0.472292078211453, 'reason': 'The Actual Output contains factual deviations such as 'Wasserzentrum' instead of 'und Wasser' and omits 'in neuer'.'}","{'score': 0.404512953793764, 'reason': 'Output has grammatical issues: 'Siso's' should be 'Sisos' and 'Wasserzentrum neue Galerie Ausstellung' is incorrect structure and word choice compared to expected 'und Wasser in neuer Ausstellung'.'}"
1,"""People Swimming in the Swimming Pool"" from 2022 is one Vicente Siso artwork that will display at Tierra del Sol Gallery beginning Jan. 13. (photo courtesy of Vicente Siso)","„People Swimming in the Swimming Pool“ aus dem Jahr 2022 ist ein Kunstwerk von Vicente Siso, das ab dem 13. Januar in der Galerie „Tierra del Sol“ ausgestellt wird (Foto bereitgestellt von Vicente Siso)","""People Swimming in the Swimming Pool"" von 2022 ist ein Vicente Siso Kunstwerk, das in der Tierra del Sol Gallery ab dem 13. Januar ausgestellt wird. (Foto mit freundlicher Genehmigung von Vicente Siso)","{'score': 0.8679178692681611, 'reason': 'The actual output closely matches the expected output with minor differences: the use of 'von' instead of 'aus dem Jahr' for 'from' and 'mit freundlicher Genehmigung von' instead of 'bereitgestellt von' for 'photo courtesy of'. These do not affect the factual content significantly.'}","{'score': 0.726302293135876, 'reason': 'The translation is mostly clear but has some issues such as 'ein Vicente Siso Kunstwerk' instead of 'ein Kunstwerk von Vicente Siso' and lack of quotes around 'Tierra del Sol'. These impact clarity slightly but the main content remains comprehensible.'}"


### Models Endpoint
- List models supported for task(s)

In [18]:
### Get the list of models supported for translation
models_response = client.models.get_suggested_models(tasks=["translation"])
for model_config in models_response.items:
    print(model_config.model)

facebook/m2m100_418M
facebook/m2m100_1.2B
Helsinki-NLP/opus-mt
bigscience/mt0-base
bigscience/mt0-large
bigscience/mt0-xl
gpt-4o-mini
gpt-4o
deepseek-reasoner
deepseek-chat
ministral-8b-latest
mistralai/Mistral-7B-Instruct-v0.2
